# Quantum Nearest Neighbors


In this [repository](https://github.com/andre-juan/quantum_nearest_neighbors), I implement the algorithm introduced in [this paper](https://arxiv.org/pdf/1412.3646.pdf) by [Maria Schuld](https://scholar.google.com/citations?user=_ih_hwUAAAAJ&hl=en), [Ilya Sinayskiy](https://scholar.google.co.za/citations?user=tL1_WfsAAAAJ&hl=en) and [Francesco Petruccione](https://scholar.google.com/citations?user=chM4fT4AAAAJ&hl=en).


The algorithm is proposed to solve **classification problems**: assign one out of a number of discrete classes to an observation, according to a rule learned from a set of labeled (classified) examples. Some examples of such problems are:

- Diagnosing a disease given a number of symptoms;
- Credit decision based on probability of default;
- An email being automatically marked as \'important\' or \'spam\';
- A handwritten digit on a postal envelope being recognised by a scanning device. 

We will be working in the **supervised learning** paradigm:

- We have a set of $n$-dimensional data vectors $\{\vec{v}^p\}$;
  
- And their respective class assignments $c^p$. Classes are often encoded by a finite number $d$ of positive integers $c\in\{1,...,d\}$. A particular case of interest is that of **binary classification**, in which case $d=2$, and $c\in\{0, 1\}$;
  
- $\mathcal{T}=\{ (\vec{v}^{p}, c^p) \}_{p=1,\cdots, N}$ makes up the training set.
  
  
Each one of the training vectors encodes $n$ *features* (or *predictors*) $v_{i}^p$, $i=1,\cdots, n$.  Features may be given by binary, integer or real-valued numbers.
  
 
In the supervised learning paradign, we can schematically see a classification problem as:

> Given: 
> - Training set $\mathcal{T}=\{ (\vec{v}^{p}, c^p) \}_{p=1,\cdots, N}$;
> - Unclassified vector $\vec{x}$, encoding $n$ features. 

  
> Goal: 
> - Match the input vector $\vec{x}$ to a class, using information from the training data. 

  Possible approach for classification:

- Define some distance measure;
- Assign the input vector to the class whose members are most similar in terms of this distance (i.e., whose members are closest to it). 

A common distance measure is the Hamming distance, which is used when the features are encoded as **binary strings**. It is defined as the number of different bits, or components, among a pair of binary strings.

> Ex.: Take $a = 1011101$ and $b = 1001001$. $d_H(a,b) = 2$.

______________



The **quantum nearest neighbors** algorithm is a quantum classification algorithm, which uses the aforementioned distance criterion for classification, based on Hamming distance. Schematically, the algorithm goes like:
  
- We first create a superposition of the training data set;
  
- Then write the Hamming distance between the input observation and each example in the training set into the amplitude of each vector in the superposition;
  
- Measuring the class-qudit then retrieves the desired class with the highest probability. 

Step by step, we have:

___________________


**STEP 0 - state preparation**

- First, extract the features of the datapoints in the training set and store them as bit strings/bit vectors. This is done classically, via some preprocessing routine.

- Then map these bit vectors to quantum computational-basis states: $0 \mapsto \left | 0 \right >$ and $1 \mapsto \left | 1 \right >$.

After these two steps, each training set datapoint is mapped to the quantum state $\left | v_1^p \cdots v_n^p \right > \equiv \left | \vec{v}^p \right >$, $v_k^p \in \{0,1\} $, $p = 1, \cdots, N$.

Consider also the class $c^p \in \{1, \cdots, d\}$, and construct the state:
  
$\left | v_1^p \cdots v_n^p, c^p \right > \equiv \left |\vec{v}^p, c^p\right > \in \mathcal{H}_{2}^{\otimes n} \otimes \mathcal{H}_d \$.

If we're dealing with binary classification, the classes are also straightforwardly encoded in qubits as $0 \mapsto \left | 0 \right >$ and $1 \mapsto \left | 1 \right >$. If we have a multiclass problem, qudits are necessary, ore one could use qubits to encode integers corresponding to the classes (for instance, $\left | 011 \right > = \left | 3 \right >$) 

Once we have the training states  $\{\left |\vec{v}^p, c^p\right > \}\in \mathcal{H}_{2}^{\otimes n} \otimes \mathcal{H}_d$, $p = 1,...,N$, we construct a training set superposition of all datapoints,
  
$\left |T\right > = \frac{1}{\sqrt{N}} \sum_{p=1}^N \left |\vec{v}^p, c^p\right >.$
  
Now, do the same binarization process with the unclassified n-dimensional input vector $\vec{x}$, and map it to the feature vector $ \left |x_1 \cdots x_n\right > \equiv \left |\vec{x}\right >$, $x_k \in \{0,1\}$.

Finally, add an ancilla register $\left | 0 \right >$ as the last register.

From this, we construct the initial state:
  
$\left |\psi_0\right > = \frac{1}{\sqrt{N}} \sum_{p=1}^N \left |\vec{x};\vec{v}^p, c^p ; 0\right > \ ,$
 
which is made of three registers: 

- The first containing the input state $\left |\vec{x}\right >$;
- The second  containing the superposition $\left |T\right >$ (which is the tensor product of the feature vectors $\left |\vec{v}^p\right >$ and the class vectors $\left |c^p\right >$); 
- The third containing an ancilla qubit set to $\left |0\right >$. 

Once the initial state is prepared, the proper operations of the algorithm begin!

___________________


**STEP 1**

The ancilla is put into a superposition,
  
$ \left |\psi_1\right > = \frac{1}{\sqrt{N}} \sum_{p=1}^N \left |\vec{x};\vec{v}^p, c^p \right > \otimes \frac{1}{\sqrt{2}} (\left |0\right >+ \left |1\right >). $

___________________


**STEP 2**

The Hamming distance between each qubit of the first (input) and second (training) register replaces the qubits in the second register, 
  
$ d_k^{i} = \left\{   \begin{array}{l l}
		      0, & \quad \mathrm{if} \; \left |v_k^p\right > = \left |x_k\right >,\\
		      1, & \quad \mathrm{else,} 
		  \end{array} \right . $ 
	

To do this, apply a $\mathrm{cNOT}(x_k,v_k^p)$-gate, which overwrites the second entry $v_k^p$ with 0 if $v_k^p=x_k$ and else with $1$:
   
$ \left\{\begin{matrix}
        \mathrm{cNOT} \left |00\right > = \left |00\right > \ ; \  \mathrm{cNOT} \left |01\right > = \left |01\right >\\ 
        \mathrm{cNOT} \left |11\right > = \left |10\right > \ ; \ \mathrm{cNOT} \left |10\right > = \left |11\right >
        \end{matrix}\right. $

Thus, the second step accounts to
  
$\left |\psi_2\right > = \prod_{k=1}^n \mathrm{cNOT}(x_k, v^p_k) \; \left |\psi_1\right > = \frac{1}{\sqrt{N}}\sum_{p=1}^N \left |\vec{x};\vec{d}^p, c^p\right > \otimes \frac{1}{\sqrt{2}}(\left |0\right > + \left |1\right >)
  ,$
  
where the Hamming distance components $\left |d_1^p \cdots d_n^p\right > \equiv \left |\vec{d}^p\right >$, $d_k^p \in \{0,1\}$, $p = 1, \cdots, N$ are now in the second register.

___________________


**STEP 3**

Use the unitary operator

$U = e^{ -i\frac{\pi }{2n} H}  \ ; \qquad\; H = 1 \otimes \sum_{k=1}^n \left(\frac{1-\sigma_z}{2}\right)_{d_k}  \otimes 1 \otimes \sigma_z\; , $

This sums the Hamming distance components $\{d^p_k\}$ (thus yielding the actual Hamming distance) between $\left |\vec{v}^p\right >$ and $\left |\vec{x}\right >$, $d_H(\vec{x},\vec{v}^p) \equiv d_H$, into the phase of the $p$th state of the superposition.

A negative sign is added, depending on the ancilla state. 

The state after the third step is:

$\left |\psi_3\right > = U \left |\psi_2\right > = \frac{1}{\sqrt{2N}} \sum_{p=1}^N  \left ( e^{ -i\frac{\pi }{2n} d_H}  \left |\vec{x};\vec{d}^p, c^p; 0\right > + e^{i\frac{\pi }{2n} d_H} \left |\vec{x};\vec{d}^p, c^p; 1\right > \right )$

___________________

**STEP 4**

Apply another Hadamard on the ancilla, $H = 1 \otimes 1 \otimes 1 \otimes H$,

$\left |\psi_4\right > = H \left |\psi_3\right > = \frac{1}{\sqrt{N}} \sum_{p=1}^N  \left ( \cos\left (\frac{\pi d_H}{2n}\right) \left |\vec{x};\vec{d}^p, c^p;0\right > + \sin\left (\frac{\pi d_H}{2n}\right) \left |\vec{x};\vec{d}^p, c^p;1\right > \right ) $

Notice that $0 \leq d_H  \leq n \Rightarrow 0 \leq \frac{\pi d_H}{2n} \leq \frac{\pi}{2}$. Therefore,
  

- For large $d_H$, $\cos\left (\frac{\pi d_H}{2n}\right) \rightarrow 0$ and $\sin\left (\frac{\pi d_H}{2n}\right) \rightarrow 1$, so that we have higher probability of measuring $\left |1\right >$; 

- For short $d_H$, $\cos\left (\frac{\pi d_H}{2n}\right) \rightarrow 1$ and $\sin\left (\frac{\pi d_H}{2n}\right) \rightarrow 0$, so that we have higher probability of measuring $\left |0\right >$;  

That is, if the new input is far away from most training observations, we have a much higher probability to measure the ancilla in the state $\left |1\right >$, if the input is close to many observations we end up in state $\left |0\right >$. 

___________________

**STEP 5 - Measurement**

Now measure the ancilla of  $\left |\psi_4\right >$ in the computational basis. The probability of measuring $\left |0\right >$ is
  
    
$P(\left |0\right >_a) = \left | \left < 0 | \psi_4\right > \right |^2 =  \frac{1}{N} \sum_{p=1}^N  \cos^2\left (\frac{\pi d_H}{2n} \right)$
 

Rewrite $\left |\psi_4\right >$, to show that the different classes appear weighted by their member's distance to the input,
  

$\left |\psi_4\right > = \frac{1}{\sqrt{N}} \sum \limits_{c=1}^d \left |c\right > \otimes \sum \limits_{l\in c} \left ( \mathrm{cos}\left( \frac{\pi d_H}{2n}\right)  \left |\vec{x};\vec{d}^l; 0\right > +  \mathrm{sin}\left( \frac{\pi d_H}{2n}\right)  \left |\vec{x};\vec{d}^l; 1\right > \right )  , $

where $l$  runs over all training vectors classified with the label $c$.

The joint conditional probability to measure a certain class $c \in \{1,...,d\}$ provided we previously measured the ancilla in $0$ (state collapsed to $\left |\tilde{\psi}_4\right > = \left < 0 | \psi_4\right >\left |0\right >$) is
  
$P( c \mid \left |0\right >_a ) = P(c)P(\left |0\right >_a) =  \left |  \left < c | \tilde{\psi}_4 \right > \right |^2 =  \frac{1}{N} \sum \limits_{l\in c} \cos^2\left (\frac{\pi d_H}{2n} \right) \ ,$

so that: 
 
$P(c) =  \frac{1}{ P(\left |0\right >_a)}\frac{1}{N} \sum \limits_{l\in c} \cos^2\left (\frac{\pi d_H}{2n} \right) \ .$

Thus, the class measured with highest probability is that whose members are the closest to the input vector! 

And, finally, the most probable class is chosen as the final prediction!

Now, head to [the repository](https://github.com/andre-juan/quantum_nearest_neighbors), and you'll find Python/Qiskit implementations of the algorithm described above. Have fun! :D
_________________